In [102]:
import os
import cv2
import numpy as np
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler

# Klasörden resimleri okuma ve vektöre dönüştürme fonksiyonu
def load_images_from_folder_cv2(folder, img_size=(128, 128)):
    images = []  # Resimleri depolamak için liste
    labels = []  # Etiketleri depolamak için liste

    for subfolder in os.listdir(folder):  # Ana klasördeki alt klasörlere bak
        subfolder_path = os.path.join(folder, subfolder)  # Alt klasör yolu
        if os.path.isdir(subfolder_path):  # Alt klasörse işlem yap
            for filename in os.listdir(subfolder_path):  # Alt klasördeki her dosyaya bak
                img_path = os.path.join(subfolder_path, filename)  # Dosya yolu
                if filename.lower().endswith(".jpg"):  # Sadece .jpg dosyalarını al
                    img = cv2.imread(img_path)  # Resmi oku
                    if img is not None:
                        img = cv2.resize(img, img_size)  # Resmi belirtilen boyutta yeniden boyutlandır
                        img = img.flatten()  # Resmi tek boyutlu bir diziye dönüştür
                        images.append(img)  # Resmi listeye ekle

                        # Alt klasöre göre etiketi belirle
                        if subfolder.lower() == "camera":
                            labels.append(0)  # Camera için etiket
                        elif subfolder.lower() == "pizza":
                            labels.append(1)  # Pizza için etiket

    return np.array(images), np.array(labels)  # Resim ve etiketleri döndür

# Eğitim ve test veri klasörleri
train_folder = '/content/train/train'
test_folder = '/content/test/test'

# Eğitim ve test verilerini yükle
train_images, train_labels = load_images_from_folder_cv2(train_folder)
test_images, test_labels = load_images_from_folder_cv2(test_folder)

# Veri şekillerini ekrana yazdır
print("Train images shape:", train_images.shape)
print("Train labels shape:", train_labels.shape)
print("Test images shape:", test_images.shape)
print("Test labels shape:", test_labels.shape)

# Veri normalizasyonu (0-1 aralığına çekme)
scaler = MinMaxScaler()
train_images = scaler.fit_transform(train_images)  # Eğitim verilerini ölçeklendir
test_images = scaler.transform(test_images)        # Test verilerini aynı ölçekle ölçeklendir

# Ağırlıkları başlatma fonksiyonu
def initialize_weights(input_size):
    weights = np.random.randn(input_size + 1) * 0.01  # Küçük değerlerle başlat
    return weights

input_size = train_images.shape[1]  # Girdi boyutu
weights = initialize_weights(input_size)  # Ağırlıkları başlat

# Softplus aktivasyon fonksiyonu
def softplus(x):
    x = np.clip(x, -20, 20)  # Değerleri belirli bir aralıkta tut
    return np.log(1 + np.exp(x))  # Softplus fonksiyonu

# Softplus fonksiyonunun türevi
def der_softplus(x):
    x = np.clip(x, -20, 20)
    return 1 / (1 + np.exp(-x))

def trainPerceptron(inputs, t, weights, rho=0.0001, iterNo=1000):
    bias = np.ones((inputs.shape[0], 1))  # Bias sütunu
    inputs_bias = np.hstack((inputs, bias))  # Girdilere bias ekle

    for iteration in range(iterNo):
        inputs_bias, t = shuffle(inputs_bias, t)  # Verileri her iterasyonda karıştır
        total_loss = 0

        for i in range(inputs_bias.shape[0]):
            x = inputs_bias[i]
            target = t[i]

            # Feedforward işlemi
            z = np.dot(x, weights)  # Net giriş hesapla
            output = softplus(z)  # Softplus ile aktivasyon

            # Hata hesaplama
            error = target - output  # Hata hesapla
            total_loss += error**2

            # Geri yayılım ve ağırlık güncelleme
            gradient = error * der_softplus(z) * x  # Gradyan hesapla
            weights += rho * gradient  # Ağırlıkları güncelle

        # Ağırlıkları sınırlı bir aralıkta tut
        weights = np.clip(weights, -1, 1)

    np.save('weights.npy', weights)  # Ağırlıkları kaydet
    return weights

# Perceptron modelini eğit (class_weight olmadan)
trained_weights = trainPerceptron(train_images, train_labels, weights, rho=0.0001)

print("Eğitim tamamlandı ve ağırlıklar kaydedildi.")

# Perceptron modelini test etme fonksiyonu
def testPerceptron(sample_test, weights):
    bias = np.ones((sample_test.shape[0], 1))
    sample_test_bias = np.hstack((sample_test, bias))

    # Feedforward işlemi
    z = np.dot(sample_test_bias, weights)
    outputs = softplus(z)

    # Çıkışları sınıflandırmak için 0.5 eşik değeri kullanıyoruz
    predicted_labels = (outputs >= 0.5).astype(int)
    return predicted_labels

# Test verileriyle tahmin yap
weights = np.load('weights.npy')
predicted_labels = testPerceptron(test_images, weights)

for i in range(len(predicted_labels)):
    print(f"Test Sample {i+1}: Predicted class = {predicted_labels[i]}, Actual class = {test_labels[i]}")

# Genel doğruluğu hesapla
def calculate_accuracy(predicted_labels, true_labels):
    accuracy = np.mean(predicted_labels == true_labels)
    return accuracy

# Doğruluğu ekrana yazdır
accuracy = calculate_accuracy(predicted_labels, test_labels)
print("Test doğruluğu: {:.2f}%".format(accuracy * 100))

Train images shape: (82, 49152)
Train labels shape: (82,)
Test images shape: (11, 49152)
Test labels shape: (11,)
Eğitim tamamlandı ve ağırlıklar kaydedildi.
Test Sample 1: Predicted class = 1, Actual class = 1
Test Sample 2: Predicted class = 0, Actual class = 1
Test Sample 3: Predicted class = 1, Actual class = 1
Test Sample 4: Predicted class = 0, Actual class = 1
Test Sample 5: Predicted class = 0, Actual class = 1
Test Sample 6: Predicted class = 1, Actual class = 1
Test Sample 7: Predicted class = 0, Actual class = 0
Test Sample 8: Predicted class = 0, Actual class = 0
Test Sample 9: Predicted class = 0, Actual class = 0
Test Sample 10: Predicted class = 0, Actual class = 0
Test Sample 11: Predicted class = 0, Actual class = 0
Test doğruluğu: 72.73%
